## vehicles_id_price에 성능점검기록 추가해서 data.csv에 저장하고 error난 경우 error_ids.csv에 저장.
이미 검증한 id들을 저장 후 확인하기때문에 중복검증 X

In [ ]:
from tqdm import tqdm
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 기존 데이터 불러오기
try:
    data_df = pd.read_csv('data.csv') # 데이터 추출 저장 완료한 csv
    existing_ids = set(data_df['id'])  # 이미 존재하는 id 리스트
except FileNotFoundError:
    existing_ids = set()  # 파일이 없으면 빈 집합

# 기존 에러 id 데이터 불러오기
try:
    data_df = pd.read_csv('error_ids.csv')
    error_ids = set(data_df['id'])  # 이미 존재하는 id 리스트
except FileNotFoundError:
    error_ids = set()  # 파일이 없으면 빈 집합

# vehicles_id 데이터 읽기
vehicles_df = pd.read_csv('vehicles_id_price.csv')

# vehicles_id와 price 데이터를 딕셔너리 형태로 저장 (ID를 키, 가격을 값으로 설정)
price_dict = pd.Series(vehicles_df['price'].values, index=vehicles_df['id']).to_dict()

# 상위 10,000개의 차량 ID를 리스트로 가져옴
all_numbers = vehicles_df['id'][:10000].tolist()  # vehicles_id 열에서 차량 ID 리스트를 가져옴

# 중복되지 않은 차량 ID만 필터링 (기존 데이터와 오류 ID를 제외)
remaining_numbers = [car_id for car_id in all_numbers if car_id not in existing_ids and car_id not in error_ids]

# 크롬 드라이버 설정
options = webdriver.ChromeOptions(
options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()), options=options)

# 차량 정보 수집 후 데이터를 추가할 리스트
new_data = []
# 에러 ID를 저장할 리스트 / 에러 Id : 보험이력, 성능점검 둘중 하나라도 없는 경우
new_error = []  
#데이터를 저장할 배치 크기 저장 / 메모리 부족 문제 해결 or 예상치 못한 에러 발생 대비
batch_size = 100 

# 헤더 정의
# header = [
#     'Car ID', '가격', '차량번호', '최초등록일', '변속기종류', '사용연료', '주행거리(km)', '주행거리 계기상태', 
#         '차대번호표기', '튜닝', '특별이력', '용도변경', '배출가스', '리콜대상', '차명', '연식'
# ]
header = [
    'id', 'price', 'car_number', 'registration_date', 'transmission', 'fuel_type', 'mileage', 'mileage_condition', 
    'chassis_number', 'tuning', 'special_history', 'usage_change', 'emission_gas', 'recall', 'car_name', 'year'
]

try:
  # 저장안된 차량 id를 대상으로 크롤링
  for idx, car_id in enumerate(tqdm(remaining_numbers), start=1):
      if car_id in existing_ids:
          continue  # 이미 존재하는 ID는 건너뜁니다.
          
      url = f"https://fem.encar.com/cars/report/inspect/{car_id}" #성능점검 페이지
      driver.get(url)
      try:
          WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/ul[2]/li[1]/span[2]")))
          
          car_number = driver.find_element(By.XPATH, "//div[@class='ReportInspectList_car_inspect__wmdMG']/strong").text.split(', ')[1].strip()
          registration_date =  driver.find_element(By.XPATH, "//div[@style='padding-bottom: 130px;']/strong").text.split(' ')[-1].strip(')')
          
          transmission = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O']/li[1]/span[2]").text.split(' ')[-1].strip(')')
          fuel_type = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O']/li[2]/span[2]").text
          mileage_condition = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[1]/span[2]").text
          mileage = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[2]/span[2]").text
          chassis_number = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[3]/span[2]").text
          tuning = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[4]/span[2]").text
          special_history = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[5]/span[2]").text
          usage_change = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[6]/span[2]").text
          emission_gas = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[7]/span[2]").text
          recall = driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[8]/span[2]").text

          # 가격 정보 가져오기 (vehicles_id_price.csv에서 가져옴)
          price = price_dict.get(car_id, '가격 없음')  # 가격이 없으면 '가격 없음'으로 설정

          # 보험이력 페이지
          url = f"https://fem.encar.com/cars/report/accident/{car_id}"
          driver.get(url)

          WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[2]/dl/dd/div/table/tbody/tr")))
          car_name = driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[1]/span/span[2]").text
          year = driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[1]/span/span[3]").text

          # 수집한 데이터를 new_data 리스트에 추가
          new_data.append([
              car_id, price, car_number, registration_date, transmission, fuel_type, mileage, mileage_condition, 
              chassis_number, tuning, special_history, usage_change, emission_gas, recall, car_name, year
          ])

      except Exception as e:
          new_error.append(car_id)

      # 배치 크기마다 데이터를 저장
      if len(new_data) % batch_size == 0 and len(new_data) != 0:
          # data.csv 파일이 없으면 새로 만들고 헤더 추가
          with open('data.csv', 'a', newline='', encoding='utf-8') as file:
              writer = csv.writer(file)
              if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
                  writer.writerow(header)
              writer.writerows(new_data)
          new_data = []  # 저장 후 리스트 초기화
          print("데이터가 data.csv에 추가되었습니다.")
      
      if len(new_error) % 10 == 0 and len(new_error)!=0:
          with open('error_ids.csv', 'a', newline='', encoding='utf-8') as file:
              writer = csv.writer(file)
              if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
                  writer.writerow(['id'])  # 'id'만 포함된 헤더 작성
              for error_id in new_error:  # 반복문으로 ID 추가
                  writer.writerow([error_id])
          new_error = [] #저장 후 리스트 초기화

except KeyboardInterrupt: #사용자가 스크립트를 중간에 멈출시
  if new_data:  # new_data에 데이터가 남아 있는 경우
      with open('data.csv', 'a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file)
          if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
              writer.writerow(header)
          writer.writerows(new_data)
      print("남은 데이터가 data.csv에 추가되었습니다.")

  if new_error:
      with open('error_ids.csv', 'a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file)
          if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
              writer.writerow(['id'])  # 'id'만 포함된 헤더 작성
          for error_id in new_error:  # 반복문으로 ID 추가
              writer.writerow([error_id])

else:
  # 반복문 종료 후 남은 데이터 저장
  if new_data:  # new_data에 데이터가 남아 있는 경우
      with open('data.csv', 'a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file)
          if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
              writer.writerow(header)
          writer.writerows(new_data)
      print("남은 데이터가 data.csv에 추가되었습니다.")

  if new_error:
      with open('error_ids.csv', 'a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file)
          if file.tell() == 0:  # 파일이 비어 있으면 헤더 추가
              writer.writerow(['id'])  # 'id'만 포함된 헤더 작성
          for error_id in new_error:  # 반복문으로 ID 추가
              writer.writerow([error_id])

# 크롬 드라이버 종료
driver.quit()


  0%|          | 31/10000 [01:04<5:47:25,  2.09s/it]


남은 데이터가 data.csv에 추가되었습니다.
